In [1]:
import csv
import pandas as pd

def csv_to_triples(csv_file, relationship_mapping, output_csv, combined_column_mappings):
    triples = []
    
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader) 
        
        for row in reader:
            subject = row[0].strip()  #First column as subject
            
            for i in range(1, len(row)):
                column_name = headers[i].strip()
                if column_name in relationship_mapping:  #Skip columns not in mapping
                    relationship = relationship_mapping[column_name]
                    obj = row[i].strip()
                    if obj:
                        triples.append((subject, relationship, obj))
            
            for combined_columns, combined_relationship in combined_column_mappings:

                combined_values = [str(row[headers.index(col)].strip()) for col in combined_columns if row[headers.index(col)].strip()]
                
                if combined_values:
                    combined_obj = ', '.join(combined_values)
                
                    #Check if it's the "is_open_from" relationship and all values are "?"
                    if combined_relationship == "is_open_from" and combined_obj == "?, ?, ?, ?, ?, ?, ?":
                        combined_obj = "We are unsure when they are open"
                
                    triples.append((subject, combined_relationship, combined_obj))
    
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Subject", "Relationship", "Object"]) 
        writer.writerows(triples)
    
    return triples

#Custom relationships
relationship_mapping = {
    "Main_Services": "main_services_are",
    "Other_Services": "other_services_are",
    "Phone_Number": "phone_number_is",
    "Website": "website_is",
    "Location_Address": "is_located_at",
    "Location_URL_Map": "url_is_located_at",
    "Availability": "has_an_availability_status_of",
    "Description": "description_is",
    "Languages": "speaks",
    "Cost": "costs",
    "Google_Review": "has_Google_Reviews",
    "Coverage": "covers",
    "Zipcode": "zipcode_is",
    "24hour": "24hours_status_is",
    "Monday": "Monday's_time_open",
    "Tuesday": "Tuesday's_time_open",
    "Wednesday": "Wednesday's_time_open",
    "Thursday": "Thursday's_time_open",
    "Friday": "Friday's_time_open",
    "Saturday": "Saturday's_time_open",
    "Sunday": "Sunday's_time_open",
    "Google_Rating": "has_a_Google_Rating_of",
    "Service_Type": "offers"
}

#Multiple column combinations and their respective relationships
combined_column_mappings = [
    (["Serving", "Eligibility"], "serves"),
    (["Facebook_URL", "Twitter_URL"], "socials_are"),
    (["Latitude", "Longitude"], "coordinates_are")
]

csv_file_path = 'Leo/Riverside/FindHelp_extracted_data_riv_shelter.csv' #Change to your path
output_csv = "Riv_shelter_triples.csv"  #Change to yours
triples = csv_to_triples(csv_file_path, relationship_mapping, output_csv, combined_column_mappings)

df = pd.read_csv("Riv_shelter_triples.csv") #Make sure matches the output_csv

df.to_csv("Riv_shelter_triples.csv", index=False, encoding='utf-8')

df

,Subject,Relationship,Object
0,Disaster Relief And Recovery (American Red Cro...,main_services_are,"['disaster response', 'emergency food', 'tempo..."
1,Disaster Relief And Recovery (American Red Cro...,other_services_are,"['psychiatric emergency services', 'mental hea..."
2,Disaster Relief And Recovery (American Red Cro...,phone_number_is,888-831-0031
3,Disaster Relief And Recovery (American Red Cro...,website_is,https://www.redcross.org/local/california/sout...
4,Disaster Relief And Recovery (American Red Cro...,is_located_at,"6235 South Riverside Avenue, Colton, CA 92507"
...,...,...,...
284,Safe Haven Welcome Centers (RI International -...,Sunday's_time_open,Closed
285,Safe Haven Welcome Centers (RI International -...,has_a_Google_Rating_of,0
286,Safe Haven Welcome Centers (RI International -...,offers,Shelter
287,Safe Haven Welcome Centers (RI International -...,serves,"['adults 18+', 'homeless', 'near homeless', 'a..."
